# Machine Learning - Training the models using a LSTM based network

First we are importing the necessary libraries and the datasets

In [4]:
import tensorflow as tf

from numpy import genfromtxt
import pandas as pd
from sklearn import preprocessing, model_selection
from tensorflow import keras
from tensorflow.metrics import precision
import matplotlib.pyplot as plt 
import os
from sklearn.metrics import precision_recall_fscore_support

features = genfromtxt('../datasets/final-data/features.csv', delimiter=',')
labels = genfromtxt('../datasets/final-data/labels.csv', delimiter=',')
#embedding_matrix = genfromtxt('../datasets/final-data/embedding_matrix.csv', delimiter=',')

train_x, test_x, train_y, test_y = model_selection.train_test_split(features,labels,test_size = 0.3, random_state = 0)

In [5]:
train_x.shape

(2982, 900)

# Creating the models to be tested

Now we are combining all the variable parameters and creating multiple models to be tested

In [6]:
# input_dim = len(embedding_matrix)
# input_length = len(train_x[0])

# Variable parameters to be tested
units_options = [150, 75, 35]
activation_functions = ["relu", "sigmoid", "tanh"]
learning_rates = [0.1, 0.01, 0.001, 0.0001]
momentum_options = [0.1, 0.01, 0.001, 0.0001]
decay_options = [0.1, 0.01, 0.001, 0.0001]

#Create multiple models combining all the parameters
models = []
for units in units_options:
    for learning_rate in learning_rates:
        for momentum in momentum_options:
            for activation_function in activation_functions:
                for decay in decay_options:

                    #Creating the network structure
                    model = keras.Sequential()

                    # Input, 
                    # Dense(linear(units = 150)), 
                    # Dense(relu), 
                    # batch_normalization(opcional, BatchNormalization1D), 
                    # linear(2)

                    model.add(
                        keras.layers.Input(
                            shape=900,
                            sparse=False
                        )
                    )


                    model.add(keras.layers.Dense(450))
                    model.add(keras.layers.Dense(units, activation = activation_function))
                    model.add(keras.layers.BatchNormalization())
                    model.add(keras.layers.Dense(2, activation='softmax'))

                    # Setting the optimizers parameters
                    optimizer = tf.keras.optimizers.SGD(
                        learning_rate=learning_rate,
                        decay=decay,
                        momentum=momentum,
                        nesterov=True
                    )

                    # Compiling the model
                    model.compile(
                        optimizer = optimizer,
                        loss='sparse_categorical_crossentropy',
                        metrics=['acc', 'mae', 'mse'])

                    # Including the new model in the models array
                    models.append(
                        {
                            "model_name": 'model_' + str(len(models) + 1),
                            "units": units, 
                            "learning_rate": learning_rate, 
                            "momentum": momentum,
                            "decay": decay,
                            "activation_function": 'relu',
                            "model": model
                        }
                    )


pd.DataFrame(models) 

,activation_function,decay,learning_rate,model,model_name,momentum,units
0,relu,0.1000,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_1,0.1000,150
1,relu,0.0100,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_2,0.1000,150
2,relu,0.0010,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_3,0.1000,150
3,relu,0.0001,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_4,0.1000,150
4,relu,0.1000,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_5,0.1000,150
5,relu,0.0100,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_6,0.1000,150
6,relu,0.0010,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_7,0.1000,150
7,relu,0.0001,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_8,0.1000,150
8,relu,0.1000,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_9,0.1000,150
9,relu,0.0100,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_10,0.1000,150


# Training the models

Training all the created models and storing their performances

In [7]:
import numpy as np
training_results = []

trained_models = 1

for model_data in models:
    model = model_data["model"]

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

    history = model.fit(
        train_x, 
        train_y, 
        epochs = 50, 
        validation_split=0.3, 
        batch_size = 16,  
        verbose=1, 
        shuffle=True,
        callbacks=[early_stop]
    )

    loss, acc, mae, mse = model.evaluate(test_x, test_y, verbose=1)
    
    test_output_probabilities = model.predict(
        test_x,
        batch_size=16,
        verbose=1,
        steps=None,
        callbacks=None,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=False
    )
    
    predicted_y = np.argmax(test_output_probabilities, axis=1)
    
    precision, recall, fs_score, support = precision_recall_fscore_support(
        y_true = test_y, 
        y_pred = predicted_y, 
        average = 'binary'
    )

    training_results.append(
        {
            "model_name": model_data["model_name"],
            "units": model_data["units"], 
            "learning_rate": model_data["learning_rate"], 
            "momentum": model_data["momentum"],
            "decay": model_data["decay"],
            "activation_function": model_data["activation_function"],
            "model": model,
            "history": history,
            "acc": acc,
            "loss": loss,
            "mae": mae,
            "mse": mse,
            "precision": precision,
            "recall": recall,
            "fs_score": fs_score,
            "test_output_probabilities": test_output_probabilities,
            "test_y": test_y,
            "predicted_y": predicted_y
        }
    )
    print("Trained models: {}".format(trained_models))
    trained_models = trained_models + 1

Train on 2087 samples, validate on 895 samples
Epoch 1/50
2087/2087 [==============================] - 19s 9ms/sample - loss: 0.7493 - acc: 0.5261 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2767 - val_loss: 0.7075 - val_acc: 0.5061 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2584
Epoch 2/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6686 - acc: 0.5894 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2616 - val_loss: 0.7148 - val_acc: 0.5140 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2636
Epoch 3/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6617 - acc: 0.6095 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2634 - val_loss: 0.7126 - val_acc: 0.5184 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2639
Epoch 4/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6528 - acc: 0.6219 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2640 - val_lo

Epoch 5/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6540 - acc: 0.6243 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2987 - val_loss: 2.9872 - val_acc: 0.5039 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.4913
Epoch 6/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6500 - acc: 0.6272 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3004 - val_loss: 1.1033 - val_acc: 0.4860 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3660
Epoch 7/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6276 - acc: 0.6560 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3062 - val_loss: 0.8381 - val_acc: 0.5408 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3130
Epoch 8/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6252 - acc: 0.6675 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3039 - val_loss: 2.1472 - val_acc: 0.5050 - val_mean_absolute

2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6467 - acc: 0.6229 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2868 - val_loss: 0.7223 - val_acc: 0.5318 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2673
Epoch 5/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6306 - acc: 0.6411 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2902 - val_loss: 0.7389 - val_acc: 0.5240 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2784
Epoch 6/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6159 - acc: 0.6584 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2943 - val_loss: 0.7983 - val_acc: 0.5061 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3003
Epoch 7/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6003 - acc: 0.6694 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2996 - val_loss: 1.4810 - val_acc: 0.5028 - val_mean_absolute_error: 0.5

2087/2087 [==============================] - 21s 10ms/sample - loss: 0.8204 - acc: 0.4950 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2924 - val_loss: 1.2520 - val_acc: 0.5028 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.4185
Epoch 2/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.7073 - acc: 0.5381 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2688 - val_loss: 0.7088 - val_acc: 0.5028 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2624
Epoch 3/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6662 - acc: 0.6028 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2778 - val_loss: 0.8317 - val_acc: 0.4961 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3106
Epoch 4/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6411 - acc: 0.6325 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2842 - val_loss: 0.9401 - val_acc: 0.5039 - val_mean_absolute_error: 0

2087/2087 [==============================] - 4s 2ms/sample - loss: 0.5855 - acc: 0.6828 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3002 - val_loss: 0.8109 - val_acc: 0.5162 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3069
Epoch 9/50
1279/1279 [==============================] - 7s 6ms/sample
Trained models: 14
Train on 2087 samples, validate on 895 samples
Epoch 1/50
2087/2087 [==============================] - 21s 10ms/sample - loss: 0.7841 - acc: 0.5156 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2843 - val_loss: 0.8544 - val_acc: 0.5028 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3196
Epoch 2/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.7121 - acc: 0.5759 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2823 - val_loss: 0.7289 - val_acc: 0.5184 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2705
Epoch 3/50
2087/2087 [==============================] - 4s 2ms/sample - loss: 0.6822 - 

...

Epoch 2/50
2087/2087 [==============================] - 11s 5ms/sample - loss: 0.8959 - acc: 0.5069 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3230 - val_loss: 0.7038 - val_acc: 0.5106 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2548
Epoch 3/50
2087/2087 [==============================] - 11s 5ms/sample - loss: 0.8913 - acc: 0.5113 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3216 - val_loss: 0.7257 - val_acc: 0.4816 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2642
Epoch 4/50
2087/2087 [==============================] - 11s 5ms/sample - loss: 0.8822 - acc: 0.5199 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3212 - val_loss: 0.7796 - val_acc: 0.4827 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2837
Epoch 5/50
2087/2087 [==============================] - 11s 6ms/sample - loss: 0.8956 - acc: 0.5089 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3211 - val_loss: 0.8539 - val_acc: 0.4860 - val_mean_abso

Epoch 3/50
2087/2087 [==============================] - 11s 5ms/sample - loss: 0.9834 - acc: 0.4969 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3358 - val_loss: 0.8327 - val_acc: 0.5263 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3103
Epoch 4/50
2087/2087 [==============================] - 11s 6ms/sample - loss: 0.9934 - acc: 0.5031 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3353 - val_loss: 0.9055 - val_acc: 0.5285 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3282
Epoch 5/50
2087/2087 [==============================] - 11s 5ms/sample - loss: 0.9788 - acc: 0.4950 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3337 - val_loss: 0.9378 - val_acc: 0.5263 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3350
Epoch 6/50
1279/1279 [==============================] - 20s 15ms/sample
Trained models: 562
Train on 2087 samples, validate on 895 samples
Epoch 1/50
2087/2087 [==============================] - 53s 25ms/sample 

2087/2087 [==============================] - 11s 5ms/sample - loss: 0.8555 - acc: 0.5055 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3089 - val_loss: 0.8411 - val_acc: 0.4849 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3091
Epoch 6/50
2087/2087 [==============================] - 12s 6ms/sample - loss: 0.8376 - acc: 0.5093 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3079 - val_loss: 0.8506 - val_acc: 0.4883 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3115
Epoch 7/50
1279/1279 [==============================] - 19s 15ms/sample
Trained models: 566
Train on 2087 samples, validate on 895 samples
Epoch 1/50
2087/2087 [==============================] - 52s 25ms/sample - loss: 0.9177 - acc: 0.5031 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3204 - val_loss: 0.7100 - val_acc: 0.5050 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2594
Epoch 2/50
2087/2087 [==============================] - 12s 6ms/sample - loss: 0.8

Epoch 6/50
1279/1279 [==============================] - 20s 15ms/sample
Trained models: 570
Train on 2087 samples, validate on 895 samples
Epoch 1/50
2087/2087 [==============================] - 53s 25ms/sample - loss: 0.8368 - acc: 0.4940 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2986 - val_loss: 0.6975 - val_acc: 0.5073 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2520
Epoch 2/50
2087/2087 [==============================] - 11s 5ms/sample - loss: 0.8266 - acc: 0.4998 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2976 - val_loss: 0.7006 - val_acc: 0.4849 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2531
Epoch 3/50
2087/2087 [==============================] - 11s 5ms/sample - loss: 0.8197 - acc: 0.4945 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2955 - val_loss: 0.7124 - val_acc: 0.4838 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2581
Epoch 4/50
2087/2087 [==============================] - 11s 5ms/sample 

1279/1279 [==============================] - 20s 16ms/sample
Trained models: 574
Train on 2087 samples, validate on 895 samples
Epoch 1/50
2087/2087 [==============================] - 53s 25ms/sample - loss: 1.1325 - acc: 0.4911 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3545 - val_loss: 0.7316 - val_acc: 0.5061 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2681
Epoch 2/50
2087/2087 [==============================] - 11s 5ms/sample - loss: 1.1006 - acc: 0.4873 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3497 - val_loss: 0.7991 - val_acc: 0.5106 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2920
Epoch 3/50
2087/2087 [==============================] - 11s 6ms/sample - loss: 1.0616 - acc: 0.4959 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3464 - val_loss: 0.9063 - val_acc: 0.5084 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3193
Epoch 4/50
2087/2087 [==============================] - 11s 5ms/sample - loss: 1.0

# Creating a dataframe with the training metrics

We are going to store those metrics in a separate file

In [8]:
training_metrics_df = pd.DataFrame(training_results)

training_metrics_df = training_metrics_df[['model_name', 'units', 'learning_rate', 'momentum', 'decay', 'activation_function', 'acc', 'loss', 'mae', 'mse', 'precision', 'recall', 'fs_score']]

training_metrics_df.head(10)

,model_name,units,learning_rate,momentum,decay,activation_function,acc,loss,mae,mse,precision,recall,fs_score
0,model_1,150,0.1,0.1,0.1000,relu,0.530102,0.720583,0.5,0.270124,0.520642,0.712716,0.601723
1,model_2,150,0.1,0.1,0.0100,relu,0.510555,0.779906,0.5,0.294583,0.509306,0.472527,0.490228
2,model_3,150,0.1,0.1,0.0010,relu,0.516810,0.936094,0.5,0.334991,0.538462,0.208791,0.300905
3,model_4,150,0.1,0.1,0.0001,relu,0.498045,1.092652,0.5,0.372378,0.497995,0.974882,0.659236
4,model_5,150,0.1,0.1,0.1000,relu,0.509773,0.732113,0.5,0.272611,0.507788,0.511774,0.509773
5,model_6,150,0.1,0.1,0.0100,relu,0.520719,0.947323,0.5,0.342968,0.556604,0.185243,0.277974
6,model_7,150,0.1,0.1,0.0010,relu,0.523847,0.906696,0.5,0.327482,0.513699,0.824176,0.632911
7,model_8,150,0.1,0.1,0.0001,relu,0.500391,0.804674,0.5,0.297478,0.498708,0.605965,0.547130
8,model_9,150,0.1,0.1,0.1000,relu,0.525410,0.720892,0.5,0.268847,0.520833,0.588697,0.552690
9,model_10,150,0.1,0.1,0.0100,relu,0.521501,0.784976,0.5,0.296661,0.521008,0.486656,0.503247


# Saving the general experiments files

Saving the files with general data about the experiment:

- All datasets used(train_x, train_y, test_x, test_y)
- The training metrics of the models

In [9]:
np.savetxt("../experiments/pmean_title/train_x.csv", train_x, delimiter=",")
np.savetxt("../experiments/pmean_title/train_y.csv", train_y, delimiter=",")
np.savetxt("../experiments/pmean_title/test_x.csv", test_x, delimiter=",")
np.savetxt("../experiments/pmean_title/test_y.csv", test_y, delimiter=",")
training_metrics_df.to_csv('../experiments/pmean_title/training_metrics.csv', index= False, encoding='utf-8')

# Saving specific metrics of each model

For each trained model we are going to save:

- Predicted labels over the test_x
- The output probability on the test_x prediction
- The history of the training
- The trained model

In [11]:
for training_result in training_results:
    # Created the model folder
    model_folder = "../experiments/pmean_title/trained_models/" + training_result["model_name"]
    if not os.path.exists(model_folder):
        os.mkdir(model_folder)
    
    # Saves the predicted labels and the predicted probabilities on the test_x predictions
    np.savetxt(model_folder + "/predicted_y.csv", training_result["predicted_y"], delimiter=",")
    np.savetxt(model_folder + "/test_output_probabilities.csv", training_result["test_output_probabilities"], delimiter=",")
    
    # Creates a history dataframe and saves it in a file
    history_df = pd.DataFrame(training_result["history"].history)
    history_df['epoch'] = training_result["history"].epoch

    history_df.to_csv(model_folder + "/history.csv", index= False, encoding='utf-8')
    
    # Saves the trained model in a file
    training_result["model"].save(filepath=model_folder + "/model.hdf5", overwrite=True, include_optimizer=True)